## Módulo de Gestión de Imágenes y Metadatos para el Dataset PAPILA
-----------------------------------------------------------------

Este módulo implementa un sistema de gestión para el dataset PAPILA,
que contiene imágenes de fondo de ojo y metadatos clínicos para la
evaluación del glaucoma.

El dataset PAPILA (Dataset with fundus images and clinical data of both
eyes of the same patient for glaucoma assessment) proporciona datos
clínicos e imágenes de fondo de ojo de ambos ojos del mismo paciente,
junto con segmentaciones del disco óptico y la copa óptica.


In [1]:
import os
from enum import Enum
from datetime import datetime
from typing import List, Dict, Optional, Tuple, Union, Any
import numpy as np
from PIL import Image

In [2]:
class Gender(Enum):
    """Enumeración para representar el género del paciente."""
    MALE = 0
    FEMALE = 1


class DiagnosisStatus(Enum):
    """
    Enumeración para representar el estado del diagnóstico.

    Según el dataset PAPILA, los diagnósticos pueden ser:
    - HEALTHY (0): Sin glaucoma
    - GLAUCOMA (1): Con glaucoma
    - SUSPECT (2): Caso sospechoso
    """
    HEALTHY = 0
    GLAUCOMA = 1
    SUSPECT = 2


class Eye(Enum):
    """
    Enumeración para representar el ojo examinado.

    OD (Oculus Dexter): Ojo derecho
    OS (Oculus Sinister): Ojo izquierdo
    """
    RIGHT = "OD"  # Oculus Dexter (ojo derecho)
    LEFT = "OS"   # Oculus Sinister (ojo izquierdo)


class CrystallineStatus(Enum):
    """
    Enumeración para representar el estado del cristalino.

    PHAKIC (0): Paciente mantiene su cristalino
    PSEUDOPHAKIC (1): Cristalino ha sido removido quirúrgicamente
    """
    PHAKIC = 0        # El ojo mantiene su cristalino
    PSEUDOPHAKIC = 1  # El cristalino ha sido removido quirúrgicamente


In [3]:
class Point:
    """
    Clase que representa un punto en coordenadas 2D.

    Utilizada para almacenar puntos de las segmentaciones del disco óptico y la copa óptica.
    """
    def __init__(self, x: float, y: float):
        """
        Inicializa un punto con coordenadas (x, y).

        Args:
            x: Coordenada x del punto
            y: Coordenada y del punto
        """
        self.x = x
        self.y = y

    def __str__(self) -> str:
        """Representación en string de un punto."""
        return f"({self.x}, {self.y})"


In [4]:
class RefractiveError:
    """
    Clase que representa el error refractivo de un ojo.

    El error refractivo es un problema de visión que ocurre cuando la forma
    del ojo no dobla la luz correctamente, lo que resulta en una imagen borrosa.
    Puede incluir miopía, hipermetropía, presbicia y astigmatismo.
    """
    def __init__(self, sphere: float, cylinder: Optional[float] = None, axis: Optional[float] = None):
        """
        Inicializa un error refractivo.

        Args:
            sphere: Valor esférico (negativo para miopía, positivo para hipermetropía)
            cylinder: Valor del cilindro (para astigmatismo)
            axis: Eje del astigmatismo en grados
        """
        self.sphere = sphere
        self.cylinder = cylinder
        self.axis = axis

    def __str__(self) -> str:
        """Representación en string del error refractivo."""
        if self.cylinder is not None and self.axis is not None:
            return f"Sphere: {self.sphere}, Cylinder: {self.cylinder}, Axis: {self.axis}°"
        return f"Sphere: {self.sphere}"

In [5]:
class Segmentation:
    """
    Clase que representa una segmentación (contorno) del disco óptico o la copa óptica.

    Las segmentaciones son fundamentales para el análisis de glaucoma ya que permiten
    calcular la relación copa/disco (CDR), un indicador importante de la enfermedad.
    """
    def __init__(self, contour_points: List[Point], expert_id: str, type_segmentation: str):
        """
        Inicializa una segmentación con puntos de contorno.

        Args:
            contour_points: Lista de puntos que forman el contorno
            expert_id: Identificador del experto que realizó la segmentación
            type_segmentation: Tipo de segmentación ('disc' o 'cup')
        """
        self.contour_points = contour_points
        self.expert_id = expert_id
        self.type_segmentation = type_segmentation

    def calculate_area(self) -> float:
        """
        Calcula el área dentro del contorno usando el algoritmo del trapecio.

        Returns:
            Área de la segmentación
        """
        # Implementación del cálculo del área usando la fórmula del trapecio
        area = 0.0
        n = len(self.contour_points)

        for i in range(n):
            j = (i + 1) % n
            area += self.contour_points[i].x * self.contour_points[j].y
            area -= self.contour_points[j].x * self.contour_points[i].y

        return abs(area) / 2.0

In [6]:
class EyeData:
    """
    Clase que representa los datos clínicos y las imágenes asociadas a un ojo específico.

    Almacena todos los datos relacionados con un ojo, incluyendo sus métricas clínicas,
    imágenes de fondo de ojo y segmentaciones.
    """
    def __init__(
            self,
            eye_type: Eye,
            diagnosis: DiagnosisStatus,
            refractive_error: Optional[RefractiveError] = None,
            crystalline_status: Optional[CrystallineStatus] = None,
            pneumatic_iop: Optional[float] = None,
            perkins_iop: Optional[float] = None,
            pachymetry: Optional[float] = None,  # Corneal thickness
            axial_length: Optional[float] = None,
            mean_defect: Optional[float] = None
    ):
        """
        Inicializa los datos de un ojo.

        Args:
            eye_type: Tipo de ojo (derecho o izquierdo)
            diagnosis: Diagnóstico del ojo
            refractive_error: Error refractivo del ojo
            crystalline_status: Estado del cristalino
            pneumatic_iop: Presión intraocular medida con método Neumático (mmHg)
            perkins_iop: Presión intraocular medida con método Perkins (mmHg)
            pachymetry: Espesor de la córnea (μm)
            axial_length: Longitud axial del ojo (mm)
            mean_defect: Defecto medio (equivalente al VFI - Visual Field Index)
        """
        self.eye_type = eye_type
        self.diagnosis = diagnosis
        self.refractive_error = refractive_error
        self.crystalline_status = crystalline_status
        self.pneumatic_iop = pneumatic_iop
        self.perkins_iop = perkins_iop
        self.pachymetry = pachymetry
        self.axial_length = axial_length
        self.mean_defect = mean_defect

        # Imágenes y segmentaciones
        self.fundus_image: Optional[str] = None  # Ruta a la imagen
        self.disc_segmentations: List[Segmentation] = []
        self.cup_segmentations: List[Segmentation] = []

    def add_fundus_image(self, image_path: str) -> None:
        """
        Agrega una imagen de fondo de ojo.

        Args:
            image_path: Ruta a la imagen de fondo de ojo
        """
        if os.path.exists(image_path):
            self.fundus_image = image_path
        else:
            raise FileNotFoundError(f"La imagen {image_path} no existe")

    def add_segmentation(self, segmentation: Segmentation) -> None:
        """
        Agrega una segmentación al ojo.

        Args:
            segmentation: Objeto de segmentación a agregar
        """
        if segmentation.type_segmentation.lower() == 'disc':
            self.disc_segmentations.append(segmentation)
        elif segmentation.type_segmentation.lower() == 'cup':
            self.cup_segmentations.append(segmentation)
        else:
            raise ValueError("El tipo de segmentación debe ser 'disc' o 'cup'")

    def calculate_cdr(self, expert_id: str = "exp1") -> Optional[float]:
        """
        Calcula la relación copa/disco (CDR) para las segmentaciones de un experto específico.

        El CDR es un indicador importante para la evaluación del glaucoma. Un valor más
        alto de CDR puede indicar la presencia de glaucoma.

        Args:
            expert_id: ID del experto cuyas segmentaciones se utilizarán

        Returns:
            La relación copa/disco o None si faltan segmentaciones
        """
        # Buscar las segmentaciones del experto especificado
        disc_seg = next((s for s in self.disc_segmentations if s.expert_id == expert_id), None)
        cup_seg = next((s for s in self.cup_segmentations if s.expert_id == expert_id), None)

        if disc_seg and cup_seg:
            disc_area = disc_seg.calculate_area()
            cup_area = cup_seg.calculate_area()

            if disc_area > 0:
                return cup_area / disc_area

        return None

    def get_glaucoma_severity(self) -> Optional[str]:
        """
        Determina la severidad del glaucoma basada en el valor del Mean Defect.

        Returns:
            Severidad del glaucoma o None si no hay datos de MD
        """
        if self.mean_defect is None or self.diagnosis != DiagnosisStatus.GLAUCOMA:
            return None

        if -6 <= self.mean_defect < -3:
            return "Glaucoma Leve"
        elif -12 <= self.mean_defect < -6:
            return "Glaucoma Moderado"
        elif self.mean_defect < -12:
            return "Glaucoma Severo"
        else:
            return "No clasificable"

In [7]:
class Patient:
    """
    Clase que representa a un paciente con datos de ambos ojos.

    Almacena la información demográfica del paciente y los datos clínicos
    de ambos ojos (derecho e izquierdo).
    """
    def __init__(
            self,
            patient_id: str,
            age: int,
            gender: Gender,
            right_eye: Optional[EyeData] = None,
            left_eye: Optional[EyeData] = None
    ):
        """
        Inicializa un paciente con sus datos básicos.

        Args:
            patient_id: Identificador único del paciente
            age: Edad del paciente
            gender: Género del paciente
            right_eye: Datos del ojo derecho
            left_eye: Datos del ojo izquierdo
        """
        self.patient_id = patient_id
        self.age = age
        self.gender = gender
        self.right_eye = right_eye
        self.left_eye = left_eye

    def set_eye_data(self, eye_data: EyeData) -> None:
        """
        Establece los datos para un ojo específico.

        Args:
            eye_data: Datos del ojo a establecer
        """
        if eye_data.eye_type == Eye.RIGHT:
            self.right_eye = eye_data
        elif eye_data.eye_type == Eye.LEFT:
            self.left_eye = eye_data
        else:
            raise ValueError("El tipo de ojo debe ser RIGHT o LEFT")

    def get_patient_diagnosis(self) -> str:
        """
        Determina el diagnóstico general del paciente basado en ambos ojos.

        Returns:
            Diagnóstico general del paciente
        """
        if not self.right_eye and not self.left_eye:
            return "Sin datos"

        # Si cualquier ojo tiene glaucoma, el paciente tiene glaucoma
        if (self.right_eye and self.right_eye.diagnosis == DiagnosisStatus.GLAUCOMA) or \
                (self.left_eye and self.left_eye.diagnosis == DiagnosisStatus.GLAUCOMA):
            return "Glaucoma"

        # Si cualquier ojo es sospechoso, el paciente es sospechoso
        if (self.right_eye and self.right_eye.diagnosis == DiagnosisStatus.SUSPECT) or \
                (self.left_eye and self.left_eye.diagnosis == DiagnosisStatus.SUSPECT):
            return "Sospechoso"

        # Si llegamos aquí, ambos ojos están sanos o uno está sano y el otro no tiene datos
        return "Sano"

In [8]:
class PapilaDataset:
    """
    Clase que representa el conjunto de datos PAPILA completo.

    Esta es la clase principal que gestiona toda la colección de pacientes
    y proporciona métodos para cargar, filtrar y analizar los datos.
    """
    def __init__(self):
        """Inicializa un dataset PAPILA vacío."""
        self.patients: Dict[str, Patient] = {}
        self.base_dir: Optional[str] = None

    def set_base_directory(self, directory: str) -> None:
        """
        Establece el directorio base donde se encuentran los datos PAPILA.

        Args:
            directory: Ruta al directorio base
        """
        if os.path.isdir(directory):
            self.base_dir = directory
        else:
            raise NotADirectoryError(f"El directorio {directory} no existe")

    def add_patient(self, patient: Patient) -> None:
        """
        Agrega un paciente al dataset.

        Args:
            patient: Objeto paciente a agregar
        """
        self.patients[patient.patient_id] = patient

    def get_patient(self, patient_id: str) -> Optional[Patient]:
        """
        Obtiene un paciente por su ID.

        Args:
            patient_id: ID del paciente a buscar

        Returns:
            Objeto paciente si existe, None en caso contrario
        """
        return self.patients.get(patient_id)

    def remove_patient(self, patient_id: str) -> bool:
        """
        Elimina un paciente del dataset.

        Args:
            patient_id: ID del paciente a eliminar

        Returns:
            True si se eliminó correctamente, False si no existía
        """
        if patient_id in self.patients:
            del self.patients[patient_id]
            return True
        return False

    def load_from_csv(self, od_file: str, os_file: str) -> None:
        """
        Carga datos desde archivos CSV para ojos derechos e izquierdos.

        Args:
            od_file: Ruta al archivo CSV con datos del ojo derecho
            os_file: Ruta al archivo CSV con datos del ojo izquierdo
        """
        # Esta es una implementación básica que debería adaptarse
        # según el formato exacto de los archivos
        # Implementación actual omitida por brevedad
        pass

    def load_segmentations(self, directory: str) -> None:
        """
        Carga las segmentaciones desde un directorio.

        Args:
            directory: Directorio que contiene los archivos de segmentación
        """
        # Implementación omitida por brevedad
        pass

    def load_images(self, directory: str) -> None:
        """
        Carga las imágenes de fondo de ojo desde un directorio.

        Args:
            directory: Directorio que contiene las imágenes
        """
        # Implementación omitida por brevedad
        pass

    def filter_patients(self, **kwargs) -> List[Patient]:
        """
        Filtra pacientes según criterios especificados.

        Args:
            **kwargs: Criterios de filtrado como age, gender, diagnosis, etc.

        Returns:
            Lista de pacientes que cumplen con los criterios
        """
        filtered_patients = list(self.patients.values())

        for key, value in kwargs.items():
            if key == 'age_min':
                filtered_patients = [p for p in filtered_patients if p.age >= value]
            elif key == 'age_max':
                filtered_patients = [p for p in filtered_patients if p.age <= value]
            elif key == 'gender':
                filtered_patients = [p for p in filtered_patients if p.gender == value]
            elif key == 'diagnosis':
                filtered_patients = [p for p in filtered_patients
                                     if (p.right_eye and p.right_eye.diagnosis == value) or
                                     (p.left_eye and p.left_eye.diagnosis == value)]

        return filtered_patients

    def get_statistics(self) -> Dict[str, Any]:
        """
        Calcula estadísticas básicas del dataset.

        Returns:
            Diccionario con estadísticas del dataset
        """
        stats = {
            "total_patients": len(self.patients),
            "gender_distribution": {
                "male": sum(1 for p in self.patients.values() if p.gender == Gender.MALE),
                "female": sum(1 for p in self.patients.values() if p.gender == Gender.FEMALE)
            },
            "diagnosis_distribution": {
                "healthy": 0,
                "glaucoma": 0,
                "suspect": 0,
                "mixed": 0  # Cuando los ojos tienen diferentes diagnósticos
            },
            "age_stats": {
                "min": float('inf'),
                "max": float('-inf'),
                "avg": 0
            }
        }

        total_age = 0
        for patient in self.patients.values():
            total_age += patient.age
            stats["age_stats"]["min"] = min(stats["age_stats"]["min"], patient.age)
            stats["age_stats"]["max"] = max(stats["age_stats"]["max"], patient.age)

            right_diagnosis = patient.right_eye.diagnosis if patient.right_eye else None
            left_diagnosis = patient.left_eye.diagnosis if patient.left_eye else None

            if right_diagnosis == left_diagnosis:
                if right_diagnosis == DiagnosisStatus.HEALTHY:
                    stats["diagnosis_distribution"]["healthy"] += 1
                elif right_diagnosis == DiagnosisStatus.GLAUCOMA:
                    stats["diagnosis_distribution"]["glaucoma"] += 1
                elif right_diagnosis == DiagnosisStatus.SUSPECT:
                    stats["diagnosis_distribution"]["suspect"] += 1
            else:
                stats["diagnosis_distribution"]["mixed"] += 1

        if self.patients:
            stats["age_stats"]["avg"] = total_age / len(self.patients)

        return stats

In [9]:
# Ejemplo de uso:
if __name__ == "__main__":
    # Crear un dataset
    dataset = PapilaDataset()

    # Crear un paciente
    patient = Patient(
        patient_id="P001",
        age=65,
        gender=Gender.MALE
    )

    # Crear datos para el ojo derecho
    right_eye = EyeData(
        eye_type=Eye.RIGHT,
        diagnosis=DiagnosisStatus.GLAUCOMA,
        refractive_error=RefractiveError(-1.5, -0.75, 180),
        crystalline_status=CrystallineStatus.PHAKIC,
        pneumatic_iop=25.0,
        perkins_iop=24.0,
        pachymetry=545.0,
        axial_length=24.5,
        mean_defect=-8.5
    )

    # Crear datos para el ojo izquierdo
    left_eye = EyeData(
        eye_type=Eye.LEFT,
        diagnosis=DiagnosisStatus.SUSPECT,
        refractive_error=RefractiveError(-1.25),
        crystalline_status=CrystallineStatus.PHAKIC,
        pneumatic_iop=22.0,
        perkins_iop=21.0,
        pachymetry=540.0,
        axial_length=24.3,
        mean_defect=-2.5
    )

    # Asignar los ojos al paciente
    patient.set_eye_data(right_eye)
    patient.set_eye_data(left_eye)

    # Agregar el paciente al dataset
    dataset.add_patient(patient)

    # Obtener estadísticas
    stats = dataset.get_statistics()
    print(f"Total de pacientes: {stats['total_patients']}")
    print(f"Diagnóstico del paciente: {patient.get_patient_diagnosis()}")

    # La severidad del glaucoma del ojo derecho
    severity = right_eye.get_glaucoma_severity()
    print(f"Severidad del glaucoma (ojo derecho): {severity}")

Total de pacientes: 1
Diagnóstico del paciente: Glaucoma
Severidad del glaucoma (ojo derecho): Glaucoma Moderado
